In [1]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Conv1D,MaxPooling1D,Flatten
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-26 19:17:24.235098: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 19:17:24.257083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 19:17:24.257106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 19:17:24.257790: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 19:17:24.262320: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14756675203812691423
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3681550336
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13509646557237972255
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-26 19:17:25.179827: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:17:25.199387: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:17:25.199430: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:17:25.364128: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:17:25.364169: I external/local_xla/xla/stream_executor

In [3]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values
    
print(train_values.shape)
print(test_values.shape)

# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


(38751, 277)
(72794, 277)
X_train shape: (31000, 275)
X_val shape: (7751, 275)
y_train shape: (31000,)
y_val shape: (7751,)


In [4]:
# Initialize the model
cnnmodel = Sequential()

# Ensure the input_shape parameter matches the shape of X_train1
cnnmodel.add(Conv1D(396, 5, strides=1, activation='relu', input_shape=(X_train1.shape[1],X_train1.shape[2])))
cnnmodel.add(MaxPooling1D(pool_size=2, strides=2))

cnnmodel.add(Conv1D(192, 7, strides=1, activation='relu'))

cnnmodel.add(MaxPooling1D(pool_size=2, strides=2))
cnnmodel.add(Conv1D(88, 9, strides=1, activation='relu'))
cnnmodel.add(MaxPooling1D(pool_size=2, strides=2))
cnnmodel.add(Conv1D(34, 11, strides=1, activation='relu', name = 'last_conv_layer'))
cnnmodel.add(MaxPooling1D(pool_size=2, strides=2))

# Flatten the tensor output from the convolutional part of the CNN
cnnmodel.add(Flatten())

# Add fully connected layers
cnnmodel.add(Dense(30, activation='relu'))
cnnmodel.add(Dense(20, activation='relu'))
cnnmodel.add(Dense(7, activation='softmax'))  

cnnmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 271, 396)          2376      
                                                                 
 max_pooling1d (MaxPooling1  (None, 135, 396)          0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 129, 192)          532416    
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 64, 192)           0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 56, 88)            152152    
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 28, 88)            0

2024-03-26 19:17:27.113522: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:17:27.113580: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:17:27.113596: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:17:27.113714: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:17:27.113730: I external/local_xla/xla/stream_executor

In [5]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)


In [6]:
class_weights

array([0.25926669, 2.33363445, 2.33258089, 2.33363445, 2.33363445,
       2.33258089])

In [7]:
cnnmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',min_delta=0.001, restore_best_weights=True)
history = cnnmodel.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)


Epoch 1/50


2024-03-26 19:17:32.228155: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-26 19:17:32.342677: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-26 19:17:32.639292: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-26 19:17:33.046104: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f130cd31a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-26 19:17:33.046132: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-26 19:17:33.050749: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711480653.103191   96737 devic

969/969 [==============================] - 11s 9ms/step - loss: 0.2888 - accuracy: 0.9101 - val_loss: 0.1235 - val_accuracy: 0.9630
Epoch 2/50
969/969 [==============================] - 7s 7ms/step - loss: 0.1163 - accuracy: 0.9643 - val_loss: 0.0915 - val_accuracy: 0.9734
Epoch 3/50
969/969 [==============================] - 7s 7ms/step - loss: 0.0813 - accuracy: 0.9734 - val_loss: 0.0698 - val_accuracy: 0.9766
Epoch 4/50
969/969 [==============================] - 7s 7ms/step - loss: 0.0665 - accuracy: 0.9788 - val_loss: 0.0654 - val_accuracy: 0.9786
Epoch 5/50
969/969 [==============================] - 7s 7ms/step - loss: 0.0512 - accuracy: 0.9835 - val_loss: 0.0631 - val_accuracy: 0.9794
Epoch 6/50
969/969 [==============================] - 7s 7ms/step - loss: 0.0465 - accuracy: 0.9850 - val_loss: 0.0779 - val_accuracy: 0.9751
Epoch 7/50
969/969 [==============================] - 7s 7ms/step - loss: 0.0408 - accuracy: 0.9874 - val_loss: 0.0568 - val_accuracy: 0.9818
Epoch 8/50
969/9

In [8]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[    0 58951  2493  6725  2923  1277   425]


In [9]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = cnnmodel.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




2275/2275 [==============================] - 5s 2ms/step
Confusion Matrix:
[[58503     2    12   121   288    25]
 [    0  2492     0     1     0     0]
 [    9     0  6678     9    29     0]
 [   14     0     3  2889    11     6]
 [   25     0     2     1  1249     0]
 [    4     0     1     4     0   416]]
F1 Score: 0.992482352480428


In [10]:
cnnmodel.save('./models/cnnmodel.h5')
